# ML Task - Length of Stay (LoS) Prediction

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from dask.distributed import Client

client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:34497/status,
Dashboard: http://127.0.0.1:34497/status,Workers: 4
Total threads: 8,Total memory: 15.05 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33405,Workers: 4
Dashboard: http://127.0.0.1:34497/status,Total threads: 8
Started: Just now,Total memory: 15.05 GiB
Comm: tcp://127.0.0.1:42095,Total threads: 2
Dashboard: http://127.0.0.1:42467/status,Memory: 3.76 GiB
Nanny: tcp://127.0.0.1:38661,


In [3]:
import dask.dataframe as dd
import pandas as pd
import os

In [4]:
local = True

if local:
    filename = os.path.join('data', 'ICUSTAYS.csv')
else:
    filename = 'gs://bdcc-mimic-iii/ICUSTAYS.csv'

icu_stays_df = dd.read_csv(filename, parse_dates=['INTIME', 'OUTTIME'], dtype={
    
})

In [5]:
icu_stays_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [6]:
icu_stays_df.describe().compute()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,FIRST_WARDID,LAST_WARDID,LOS
count,61532.000000,61532.000000,61532.000000,61532.000000,61532.000000,61532.000000,61522.000000
mean,30766.500000,33888.605912,149954.470649,249962.710248,33.085663,32.978206,4.917972
std,17762.902719,28127.690913,28898.895904,28890.574867,19.102204,19.028609,9.638784
min,1.000000,2.000000,100001.000000,200001.000000,7.000000,7.000000,0.000100
25%,15383.750000,12047.500000,124933.750000,224935.500000,14.000000,14.000000,1.108025
50%,30766.500000,24280.500000,149911.500000,249940.000000,33.000000,33.000000,2.092250
75%,46149.250000,54191.500000,174997.250000,274972.500000,52.000000,52.000000,4.483175
max,61532.000000,99999.000000,199999.000000,299999.000000,57.000000,57.000000,173.072500


In [7]:
df_los = icu_stays_df[['HADM_ID', 'INTIME', 'OUTTIME', 'LOS']]

df_los.head(15)

,HADM_ID,INTIME,OUTTIME,LOS
0,110404,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,106296,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,188028,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,173727,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,164716,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202
5,158689,2141-04-19 06:12:05,2141-04-20 17:52:11,1.4862
6,130546,2114-06-28 22:28:44,2114-07-07 18:01:16,8.8143
7,129886,2170-10-07 11:28:53,2170-10-14 14:38:07,7.1314
8,135156,2147-11-20 09:02:23,2147-11-21 17:08:52,1.3378
9,171601,2132-10-21 21:11:46,2132-10-22 14:44:48,0.7313


In [8]:
# Statistical information about the Length of Stay (LoS)
df_los[['LOS']].describe().compute()

,LOS
count,61522.000000
mean,4.917972
std,9.638784
min,0.000100
25%,1.108025
50%,2.092250
75%,4.483175
max,173.072500


In [9]:
local = True

if local:
    filename = os.path.join('data', 'EVENTS.csv')
else:
    filename = 'gs://bdcc-mimic-iii/EVENTS.csv'

events_df = dd.read_csv(filename, parse_dates=['CHARTTIME', 'STORETIME'], dtype={
    'ROW_ID': 'float64', # int4 according to specification
    'SUBJECT_ID': 'float64', # int4 according to specification
    'HADM_ID': 'float64', # int4 according to specification
    'ICUSTAY_ID': 'float64', # int4 according to specification
    'ITEMID': 'float64', # int4 according to specification
    'CGID': 'float64', # int4 according to specification
    'VALUE': 'object', # varchar according to specification
    'VALUENUM': 'float64', # float8 according to specification
    'VALUEUOM': 'object', # varchar according to specification
    'WARNING': 'float64', # int4 according to specification
    'ERROR': 'float64', # int4 according to specification
    'RESULTSTATUS': 'object', # varchar according to specification
    'STOPPED': 'object'
})

# Consider only the relevant features
events_df = events_df[['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID']]

In [10]:
from dask_ml.preprocessing import MinMaxScaler

In [11]:
def preprocess_dataset(events_df, icu_stays_df):
    res_df = icu_stays_df[['HADM_ID', 'INTIME', 'OUTTIME', 'LOS']]

    # Add a column with the number of events for each hospital admission
    events_count_df = events_df.groupby('HADM_ID')['ROW_ID'] \
        .count() \
        .compute() \
        .to_frame() \
        .reset_index()

    events_count_df.rename(columns={'ROW_ID': '# of Events'}, inplace=True)

    res_df = dd.merge(res_df, events_count_df, on='HADM_ID') \
        .compute()

    # Add the SUBJECT_ID column
    subjects_df = events_df.groupby('HADM_ID')['SUBJECT_ID'] \
        .first() \
        .compute() \
        .to_frame() \
        .reset_index()

    res_df = dd.merge(res_df, subjects_df, on='HADM_ID')

    # Reorder the columns
    res_df = res_df[['HADM_ID', 'SUBJECT_ID', 'INTIME', 'OUTTIME', '# of Events', 'LOS']]
        
    # Sort based on INTIME
    res_df = res_df.sort_values('INTIME')

    # Normalize the '# of Events' column to the [0, 1] range
    scaler = MinMaxScaler(feature_range=(0, 1))
    res_df['# of Events'] = scaler.fit_transform(res_df[['# of Events']])
    
    # Convert the dates to floats
    dt = res_df['INTIME'].values
    res_df['INTIME'] = dt.astype(float)
    
    dt = res_df['OUTTIME'].values
    res_df['OUTTIME'] = dt.astype(float)

    res_df.dropna(inplace=True)
    
    return res_df


dataset_df = preprocess_dataset(events_df, icu_stays_df)
dataset_df

,HADM_ID,SUBJECT_ID,INTIME,OUTTIME,# of Events,LOS
51858,118464,82574.0,4.116082e+18,4.116150e+18,0.000728,0.7911
37123,161773,29156.0,4.116188e+18,4.117042e+18,0.015665,9.8777
12708,173927,12001.0,4.116632e+18,4.117031e+18,0.006997,4.6201
24284,159656,21081.0,4.116667e+18,4.116764e+18,0.001019,1.1269
40193,158366,32096.0,4.117329e+18,4.117527e+18,0.005019,2.2924
...,...,...,...,...,...,...
8489,183768,7632.0,7.523203e+18,7.523675e+18,0.006979,5.4697
57836,116667,98185.0,7.530498e+18,7.530644e+18,0.002673,1.6906
3150,195990,2846.0,7.545567e+18,7.545711e+18,0.002088,1.6696
33856,119787,25115.0,7.558955e+18,7.559028e+18,0.001253,0.8452


## Prediction

In [12]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

In [13]:
import joblib
import numpy as np

In [14]:
# K-Fold with 10 folds for cr# K-Fold with 10 folds for cross-validatitrain_rowson 
cv = KFold(n_splits=10, random_state=None, shuffle=True)

In [15]:
# Split the data into training and testing data

training_size = int(.8 * len(dataset_df))

training_df = dataset_df.iloc[:training_size, :]
training_df

,HADM_ID,SUBJECT_ID,INTIME,OUTTIME,# of Events,LOS
51858,118464,82574.0,4.116082e+18,4.116150e+18,0.000728,0.7911
37123,161773,29156.0,4.116188e+18,4.117042e+18,0.015665,9.8777
12708,173927,12001.0,4.116632e+18,4.117031e+18,0.006997,4.6201
24284,159656,21081.0,4.116667e+18,4.116764e+18,0.001019,1.1269
40193,158366,32096.0,4.117329e+18,4.117527e+18,0.005019,2.2924
...,...,...,...,...,...,...
1336,139077,808.0,6.669882e+18,6.670141e+18,0.003846,2.9937
25959,162057,25031.0,6.669952e+18,6.670153e+18,0.004884,2.3174
47087,181975,53470.0,6.669955e+18,6.670043e+18,0.001040,1.0104
37002,139497,29633.0,6.669991e+18,6.670072e+18,0.001092,0.9451


In [16]:
testing_df = dataset_df.iloc[training_size:, :]
testing_df

,HADM_ID,SUBJECT_ID,INTIME,OUTTIME,# of Events,LOS
53295,148865,70466.0,6.670043e+18,6.670143e+18,0.001764,1.1558
56843,187736,97714.0,6.670054e+18,6.670143e+18,0.001154,1.0305
42693,149776,28699.0,6.670083e+18,6.670234e+18,0.001092,1.7540
14475,180730,12819.0,6.670084e+18,6.670246e+18,0.002987,1.8745
40565,187307,40236.0,6.670139e+18,6.670590e+18,0.008699,5.2128
...,...,...,...,...,...,...
8489,183768,7632.0,7.523203e+18,7.523675e+18,0.006979,5.4697
57836,116667,98185.0,7.530498e+18,7.530644e+18,0.002673,1.6906
3150,195990,2846.0,7.545567e+18,7.545711e+18,0.002088,1.6696
33856,119787,25115.0,7.558955e+18,7.559028e+18,0.001253,0.8452


In [17]:
# Separate the features (X) from the target variable (y)

X_train_vals = training_df.iloc[:, :-1].values
y_train_vals = training_df.iloc[:, -1].values

In [18]:
X_test_vals = testing_df.iloc[:, :-1].values
y_test_vals = testing_df.iloc[:, -1].values

### Linear Regression

In [19]:
scores = []

with joblib.parallel_backend('dask'):
    for train, test in cv.split(X_train_vals):
        X_train, X_test, y_train, y_test = X_train_vals[train], \
                                           X_train_vals[test], \
                                           y_train_vals[train], \
                                           y_train_vals[test]
        model = LinearRegression().fit(X_train, y_train)
        scores.append(model.score(X_test, y_test))
        
print('Score for each of the folds:', end='\n\n')
for score in scores:
    print(score)
print('\n')

print('Linear Regression Mean Score: {}'.format(np.mean(np.array(scores))), end='\n\n')

print('Score for each feature:', end='\n\n')

for i in range(X_train.shape[-1]):
    print(model.coef_[i])
    
print('\n')

print('Score for unseen data: {}'.format(model.score(X_test, y_test)))

Score for each of the folds:

0.9999999999910414
0.9999999999906958
0.9999999999902431
0.9999999999908723
0.9999999999914709
0.9999999999899638
0.9999999999922707
0.9999999999911764
0.999999999990605
0.9999999999906287


Linear Regression Mean Score: 0.9999999999908967

Score for each feature:

3.2987671938548442e-12
3.001972091690919e-12
-1.1574074358773517e-14
1.1574074358674562e-14
3.2799913680755e-20


Score for unseen data: 0.9999999999906287


### Lasso 

In [20]:
from sklearn.linear_model import Lasso

In [21]:
scores = []

with joblib.parallel_backend('dask'):
    for train, test in cv.split(X_train_vals):
        X_train, X_test, y_train, y_test = X_train_vals[train], X_train_vals[test], y_train_vals[train], y_train_vals[test]
        model = Lasso(alpha=0.01).fit(X_train, y_train)
        scores.append(model.score(X_test, y_test))
        
print('Score for each of the folds:', end='\n\n')
for score in scores:
    print(score)
print('\n')

print('Lasso Regression Mean Score: {}'.format(np.mean(np.array(scores))), end='\n\n')

print('Score for each feature:', end='\n\n')

for i in range(X_train.shape[-1]):
    print(model.coef_[i])
    
print('\n')

print('Score for unseen data: {}'.format(model.score(X_test, y_test)))

Score for each of the folds:

0.516722213028511
0.4239888881628626
0.49786908657538176
0.4918235971518552
0.40981161352120654
0.4830894107114607
0.469550537861703
0.3824490364521488
0.28543597175282054
0.5720330207832648


Lasso Regression Mean Score: 0.45327733760012145

Score for each feature:

1.8881976360090663e-06
-2.352507653232393e-05
-9.041543026894786e-18
9.007059848837429e-18
337.8287353500027


Score for unseen data: 0.5720330207832648


### Random Forest Regressor

In [22]:
from sklearn.ensemble import RandomForestRegressor

In [23]:
scores = []

with joblib.parallel_backend('dask'):
    for train, test in cv.split(X_train_vals):
        X_train, X_test, y_train, y_test = X_train_vals[train], X_train_vals[test], y_train_vals[train], y_train_vals[test]
        model = RandomForestRegressor().fit(X_train, y_train)
        scores.append(model.score(X_test, y_test))
        
print('Score for each of the folds:', end='\n\n')
for score in scores:
    print(score)

print('\n')

print('Random Forest Regressor Mean Score: {}'.format(np.mean(np.array(scores))), end='\n\n')

print('Score for unseen data: {}'.format(model.score(X_test, y_test)))

Score for each of the folds:

0.6929557638519124
0.6495298815164926
0.680139925434093
0.6597688900658677
0.7288489578952984
0.6935884762160798
0.7227802740264279
0.6933295664240415
0.6714769860376655
0.6326370646406535


Random Forest Regressor Mean Score: 0.6825055786108533

Score for unseen data: 0.6326370646406536
